In [1]:
import os
import torch
from torch import nn
import sys
sys.path.append('..')
import myd2l

In [30]:
class TokenEmbedding:
    def __init__(self, embedding_name):
        self.paths = dict()
        self.paths['glove.6b.50d'] = 'E:/Datasets/glove/glove.6B.50d/'
        self.paths['glove.6b.100d'] = 'E:/Datasets/glove/glove.6B.100d/'
        self.paths['glove.42b.300d'] = 'E:/Datasets/glove/glove.42B.300d/'
        self.paths['wiki.en'] = 'E/Datasets/fastText/wiki.en/'

        self.idx_to_token, self.idx_to_vec = self._load_embedding(embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
    
    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = self.paths[embedding_name]
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            for line in f:
                # rstrip() 方法删除最后多余的空格, \t, \n
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec

        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [self.token_to_idx.get(token, self.unknown_idx) for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        
        return vecs
    
    def __len__(self):
        return len(self.idx_to_token)
        

In [31]:
glove_6b50d = TokenEmbedding('glove.6b.50d')

In [32]:
len(glove_6b50d)

400001

In [44]:
glove_6b50d.token_to_idx['beautiful'], glove_6b50d.idx_to_token[3367]

(3367, 'beautiful')

In [45]:
def knn(W, x, k):
    cos = torch.mv(W, x.reshape(-1, )) / (torch.sqrt(torch.sum(W * W, dim=1)) * torch.sqrt((x * x).sum()) + 1e-9)
    topk, topk_idx = torch.topk(cos, k=k)
    
    return topk_idx, topk

In [46]:
def get_similar_tokens(query_token, k, embed):
    topk_idx, topk = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk_idx[1:], topk[1:]):
        print(f'{embed.idx_to_token[int(i)]}: {float(c):.3f}')

In [48]:
get_similar_tokens('chip', 3, glove_6b50d)

chips: 0.856
intel: 0.749
electronics: 0.749


In [49]:
get_similar_tokens('beautiful', 3, glove_6b50d)

lovely: 0.921
gorgeous: 0.893
wonderful: 0.830


In [50]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed[[token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk_idx, _ = knn(embed.idx_to_vec, x, 1)

    return embed.idx_to_token[int(topk_idx[0])]

In [51]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

In [56]:
get_analogy('china', 'beijing', 'japan', glove_6b50d)

'tokyo'

In [57]:
get_analogy('uncle', 'aunt', 'boy', glove_6b50d)

'girl'

In [61]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'

In [62]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'